# Logistic Regression 

Logistic regression is a statistical method used for binary classification problems, where the goal is to predict which of two classes a given observation belongs to. Despite its name, logistic regression is actually a classification algorithm, not a regression technique.


### Model Structure:

Logistic regression models the probability that a given input belongs to a particular class. This probability is estimated using the logistic function (or sigmoid function), which maps any real-valued number into a range between 0 and 1.

### Sigmoid Function: 

The core of logistic regression is the sigmoid function, defined as:
σ(z)=11+e−z
σ(z)=1+e−z1​

where z is a linear combination of the input features (i.e., z=b+∑wixiz=b+∑​wi​xi​, where b is the bias term, wi are the weights, and xi are the feature values). The output of the sigmoid function can be interpreted as the probability of the positive class.

### Training:

During training, logistic regression finds the best-fitting weights and bias that maximize the likelihood of the observed data. This is typically done using a method called maximum likelihood estimation, often implemented using optimization techniques like gradient descent.

### Decision Boundary:
Once trained, the logistic regression model uses a threshold (commonly 0.5) to classify new data. If the predicted probability is greater than or equal to the threshold, the observation is classified as belonging to the positive class; otherwise, it's classified as belonging to the negative class.###

# Logistic Regression Example

## imports

In [84]:
# imports

import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
import numpy as np


Data loading 

In [85]:
# data loading 
df = pd.read_csv('data/titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Defining X and Y target variables 

In [86]:
# defining x and y
y = df["Survived"]
X = df.drop("Survived", axis=1)

Test Train Split 

In [87]:
# test train split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

## Preproccesing 

Finding Null Values 

In [88]:
X_train.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age            133
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          511
Embarked         2
dtype: int64

Fill in categorical null values 

In [89]:
X_train_fill_na = X_train.copy()
X_train_fill_na.fillna({"Cabin":"cabin_missing", "Embarked":"embarked_missing"}, inplace=True)
X_train_fill_na.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age            133
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin            0
Embarked         0
dtype: int64

Fill in Age with mean using imputer 

In [90]:
  # Replace 'mean' with 'median', 'most_frequent', or 'constant'
imputer = SimpleImputer(strategy='mean')

In [91]:
X_train_fill_na['Age'] = imputer.fit_transform(X_train_fill_na[['Age']])
X_train_fill_na.isna().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

alternative method for recommend for multicolumn 

In [92]:



# imputer.fit(X_train_fill_na[["Age"]])
# age_imputed = pd.DataFrame(
#     imputer.transform(X_train_fill_na[["Age"]]),
#     # index is important to ensure we can concatenate with other columns
#     index=X_train_fill_na.index,
#     columns=["Age"]
# )

# X_train_fill_na["Age"] = age_imputed
# X_train_fill_na.isna().sum()

In [93]:
X_train_fill_na.info()

<class 'pandas.core.frame.DataFrame'>
Index: 668 entries, 105 to 684
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  668 non-null    int64  
 1   Pclass       668 non-null    int64  
 2   Name         668 non-null    object 
 3   Sex          668 non-null    object 
 4   Age          668 non-null    float64
 5   SibSp        668 non-null    int64  
 6   Parch        668 non-null    int64  
 7   Ticket       668 non-null    object 
 8   Fare         668 non-null    float64
 9   Cabin        668 non-null    object 
 10  Embarked     668 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 62.6+ KB


## Dealing with Categorical Varibles

selecting all Categorical variables 

In [94]:
X_train_categorical = X_train_fill_na.select_dtypes(exclude=["int64", "float64"]).copy()
X_train_categorical

,Name,Sex,Ticket,Cabin,Embarked
105,"Mionoff, Mr. Stoytcho",male,349207,cabin_missing,S
68,"Andersson, Miss. Erna Alexandra",female,3101281,cabin_missing,S
253,"Lobb, Mr. William Arthur",male,A/5. 3336,cabin_missing,S
320,"Dennis, Mr. Samuel",male,A/5 21172,cabin_missing,S
706,"Kelly, Mrs. Florence ""Fannie""",female,223596,cabin_missing,S
...,...,...,...,...,...
835,"Compton, Miss. Sara Rebecca",female,PC 17756,E49,C
192,"Andersen-Jensen, Miss. Carla Christine Nielsine",female,350046,cabin_missing,S
629,"O'Connell, Mr. Patrick D",male,334912,cabin_missing,Q
559,"de Messemaeker, Mrs. Guillaume Joseph (Emma)",female,345572,cabin_missing,S


## One Hot Encoding 

In [95]:
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)


Fit Transform

In [96]:
X_train_categorical_ohe=  ohe.fit_transform(X_train_categorical)

Convert to dataframe 

In [97]:
X_train_categorical_ohe_df = pd.DataFrame(
    X_train_categorical_ohe,
    index=X_train_categorical.index,
    columns=np.hstack(ohe.categories_)
)

X_train_categorical_ohe_df.head()

,"Abbing, Mr. Anthony","Abbott, Mr. Rossmore Edward","Abelson, Mrs. Samuel (Hannah Wizosky)","Adahl, Mr. Mauritz Nils Martin","Adams, Mr. John","Aks, Mrs. Sam (Leah Rosen)","Albimona, Mr. Nassef Cassem","Alexander, Mr. William","Alhomaki, Mr. Ilmari Rudolf","Allen, Miss. Elisabeth Walton",...,F33,F38,F4,G6,T,cabin_missing,C,Q,S,embarked_missing
105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
68,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


altenative method

In [98]:

ohe.fit(X_train_categorical)
X_train_ohe = pd.DataFrame(
    ohe.transform(X_train_categorical),
    # index is important to ensure we can concatenate with other columns
    index=X_train_categorical.index,
    # we are dummying multiple columns at once, so stack the names
    columns=np.hstack(ohe.categories_)
)
X_train_ohe

,"Abbing, Mr. Anthony","Abbott, Mr. Rossmore Edward","Abelson, Mrs. Samuel (Hannah Wizosky)","Adahl, Mr. Mauritz Nils Martin","Adams, Mr. John","Aks, Mrs. Sam (Leah Rosen)","Albimona, Mr. Nassef Cassem","Alexander, Mr. William","Alhomaki, Mr. Ilmari Rudolf","Allen, Miss. Elisabeth Walton",...,F33,F38,F4,G6,T,cabin_missing,C,Q,S,embarked_missing
105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
68,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
192,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
629,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
559,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


### Get Dummies

In [115]:
x_train_dummies =  pd.get_dummies(X_train_categorical[['Sex']],drop_first=False)
x_train_dummies.head()

,Sex_female,Sex_male
105,False,True
68,True,False
253,False,True
320,False,True
706,True,False


## Actual Categorical Features 

In [100]:
categorical_features = ["Sex", "Cabin", "Embarked"]
X_train_categorical = X_train_fill_na[categorical_features].copy()
X_train_categorical

,Sex,Cabin,Embarked
105,male,cabin_missing,S
68,female,cabin_missing,S
253,male,cabin_missing,S
320,male,cabin_missing,S
706,female,cabin_missing,S
...,...,...,...
835,female,E49,C
192,female,cabin_missing,S
629,male,cabin_missing,Q
559,female,cabin_missing,S


### Get Dummies 

In [116]:
x_train_dummies =  pd.get_dummies(X_train_categorical,drop_first=False)
x_train_dummies.head()

,Sex_female,Sex_male,Cabin_A10,Cabin_A14,Cabin_A16,Cabin_A19,Cabin_A20,Cabin_A23,Cabin_A24,Cabin_A31,...,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Cabin_cabin_missing,Embarked_C,Embarked_Q,Embarked_S,Embarked_embarked_missing
105,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
68,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
253,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
320,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
706,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False


### one hot encoder 

In [101]:
ohe.fit(X_train_categorical)

X_train_ohe = pd.DataFrame(
    ohe.transform(X_train_categorical),
    index=X_train_categorical.index,
    columns=np.hstack(ohe.categories_)
)
X_train_ohe

,female,male,A10,A14,A16,A19,A20,A23,A24,A31,...,F33,F38,F4,G6,T,cabin_missing,C,Q,S,embarked_missing
105,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
68,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
253,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
320,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
706,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
192,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
629,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
559,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [102]:
numeric_features = ["Pclass", "Age", "SibSp", "Fare"]
X_train_numeric = X_train_fill_na[numeric_features].copy()
X_train_numeric

,Pclass,Age,SibSp,Fare
105,3,28.0,0,7.8958
68,3,17.0,4,7.9250
253,3,30.0,1,16.1000
320,3,22.0,0,7.2500
706,2,45.0,0,13.5000
...,...,...,...,...
835,1,39.0,1,83.1583
192,3,19.0,1,7.8542
629,3,29.9,0,7.7333
559,3,36.0,1,17.4000


In [103]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

In [104]:
X_train_scaled = scaler.fit_transform(X_train_numeric,)
X_train_scaled

array([[1.        , 0.34451027, 0.        , 0.01541158],
       [1.        , 0.20584899, 0.5       , 0.01546857],
       [1.        , 0.36972142, 0.125     , 0.03142511],
       ...,
       [1.        , 0.36846086, 0.        , 0.0150944 ],
       [1.        , 0.44535485, 0.125     , 0.03396254],
       [0.5       , 0.74788857, 0.125     , 0.07612293]])

In [105]:
X_train_scaled_df = pd.DataFrame(
    X_train_scaled,
    index=X_train_numeric.index,
    columns=X_train_numeric.columns
)


In [106]:
X_train_scaled_df

,Pclass,Age,SibSp,Fare
105,1.0,0.344510,0.000,0.015412
68,1.0,0.205849,0.500,0.015469
253,1.0,0.369721,0.125,0.031425
320,1.0,0.268877,0.000,0.014151
706,0.5,0.558805,0.000,0.026350
...,...,...,...,...
835,0.0,0.483172,0.125,0.162314
192,1.0,0.231060,0.125,0.015330
629,1.0,0.368461,0.000,0.015094
559,1.0,0.445355,0.125,0.033963


alternative method 

In [107]:
scaler.fit(X_train_numeric)
X_train_scaled = pd.DataFrame(
    scaler.transform(X_train_numeric),
    # index is important to ensure we can concatenate with other columns
    index=X_train_numeric.index,
    columns=X_train_numeric.columns
)
X_train_scaled

,Pclass,Age,SibSp,Fare
105,1.0,0.344510,0.000,0.015412
68,1.0,0.205849,0.500,0.015469
253,1.0,0.369721,0.125,0.031425
320,1.0,0.268877,0.000,0.014151
706,0.5,0.558805,0.000,0.026350
...,...,...,...,...
835,0.0,0.483172,0.125,0.162314
192,1.0,0.231060,0.125,0.015330
629,1.0,0.368461,0.000,0.015094
559,1.0,0.445355,0.125,0.033963


In [108]:
X_train_full = pd.concat([X_train_scaled, X_train_ohe], axis=1)
X_train_full

,Pclass,Age,SibSp,Fare,female,male,A10,A14,A16,A19,...,F33,F38,F4,G6,T,cabin_missing,C,Q,S,embarked_missing
105,1.0,0.344510,0.000,0.015412,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
68,1.0,0.205849,0.500,0.015469,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
253,1.0,0.369721,0.125,0.031425,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
320,1.0,0.268877,0.000,0.014151,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
706,0.5,0.558805,0.000,0.026350,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,0.0,0.483172,0.125,0.162314,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
192,1.0,0.231060,0.125,0.015330,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
629,1.0,0.368461,0.000,0.015094,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
559,1.0,0.445355,0.125,0.033963,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


### Model defination

In [ ]:
logreg = LogisticRegression(fit_intercept=False, C=1e12, solver='liblinear')

### Model Training 

In [109]:
model_log = logreg.fit(X_train_full, y_train)
model_log

LogisticRegression(C=1000000000000.0, fit_intercept=False, solver='liblinear')

## Model results 

In [110]:
y_hat_train = logreg.predict(X_train_full)

train_residuals = np.abs(y_train - y_hat_train)
print(pd.Series(train_residuals, name="Residuals (counts)").value_counts())
print()
print(pd.Series(train_residuals, name="Residuals (proportions)").value_counts(normalize=True))

Residuals (counts)
0    567
1    101
Name: count, dtype: int64

Residuals (proportions)
0    0.848802
1    0.151198
Name: proportion, dtype: float64


### Model Validation 

In [111]:
# Filling in missing categorical data
X_test_fill_na = X_test.copy()
X_test_fill_na.fillna({"Cabin":"cabin_missing", "Embarked":"embarked_missing"}, inplace=True)

# Filling in missing numeric data
test_age_imputed = pd.DataFrame(
    imputer.transform(X_test_fill_na[["Age"]]),
    index=X_test_fill_na.index,
    columns=["Age"]
)
X_test_fill_na["Age"] = test_age_imputed

# Handling categorical data
X_test_categorical = X_test_fill_na[categorical_features].copy()
X_test_ohe = pd.DataFrame(
    ohe.transform(X_test_categorical),
    index=X_test_categorical.index,
    columns=np.hstack(ohe.categories_)
)

# Normalization
X_test_numeric = X_test_fill_na[numeric_features].copy()
X_test_scaled = pd.DataFrame(
    scaler.transform(X_test_numeric),
    index=X_test_numeric.index,
    columns=X_test_numeric.columns
)

# Concatenating categorical and numeric data
X_test_full = pd.concat([X_test_scaled, X_test_ohe], axis=1)
X_test_full

,Pclass,Age,SibSp,Fare,female,male,A10,A14,A16,A19,...,F33,F38,F4,G6,T,cabin_missing,C,Q,S,embarked_missing
495,1.0,0.368461,0.000,0.028221,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
648,1.0,0.368461,0.000,0.014737,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
278,1.0,0.079793,0.500,0.056848,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
31,0.0,0.368461,0.125,0.285990,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
255,1.0,0.357116,0.000,0.029758,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,1.0,0.558805,0.125,0.054457,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
306,0.0,0.368461,0.000,0.216430,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
379,1.0,0.231060,0.000,0.015176,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
742,0.0,0.256271,0.250,0.512122,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [112]:
y_hat_test = logreg.predict(X_test_full)

test_residuals = np.abs(y_test - y_hat_test)
print(pd.Series(test_residuals, name="Residuals (counts)").value_counts())
print()
print(pd.Series(test_residuals, name="Residuals (proportions)").value_counts(normalize=True))

Residuals (counts)
0    175
1     48
Name: count, dtype: int64

Residuals (proportions)
0    0.784753
1    0.215247
Name: proportion, dtype: float64
